# Agent Gateway Quickstart

Agent Gateway is a multi-agent framework that offers native support for Snowflake tools. 

The system can be configured to work with 3 types of tools:
- Cortex Search Tool: For unstructured data analysis, which requires a standard RAG access pattern.
- Cortex Analyst Tool: For supporting structured data analysis, which requires a Text2SQL access pattern.
- Python Tool: For supporting custom user operations (using 3rd Party API's), which requires calling arbitrary python.

This notebook walks through how to configure and run a system with all 3 types of tools. The demo is designed to illustrate how the agent can answer questions that require a divserse combination of tools (RAG,Text2SQL, Python, or a combination).

Note that Agent Gateway does not configure the underlying Cortex Search or Cortex Analyst services for the user. Those services must be configured before initializing the agent.

# Agent Configuration

## Connection Setup

Authenticate with Snowpark + set token as environment variable for use by the agents.

In [ ]:
from agent_gateway import Agent
from agent_gateway.tools import CortexSearchTool, CortexAnalystTool, PythonTool
from snowflake.snowpark import Session
import os

connection_parameters = {
    "account": os.getenv("SNOWFLAKE_ACCOUNT"),
    "user": os.getenv("SNOWFLAKE_USER"),
    "password": os.getenv("SNOWFLAKE_PASSWORD"),
    "role": os.getenv("SNOWFLAKE_ROLE"),
    "warehouse": os.getenv("SNOWFLAKE_WAREHOUSE"),
    "database": os.getenv("SNOWFLAKE_DATABASE"),
    "schema": os.getenv("SNOWFLAKE_SCHEMA"),
}

snowpark = Session.builder.configs(connection_parameters).create()

## Snowflake Tool Configuration

The Cortex Search Tool and the Cortex Analyst Tool need to be configured as follows. Note that a connection object is required for each config. In the case below we're using the same connection object for both because the services are both in the same account/database/schema. Users have the option to pass in different connection objects as needed.

In [3]:
search_config = {
    "service_name": "SEC_SEARCH_SERVICE",
    "service_topic": "Snowflake's business,product offerings,and performance.",
    "data_description": "Snowflake annual reports",
    "retrieval_columns": ["CHUNK"],
    "snowflake_connection": snowpark,
}

analyst_config = {
    "semantic_model": "sp500_semantic_model.yaml",
    "stage": "ANALYST",
    "service_topic": "S&P500 company and stock metrics",
    "data_description": "a table with stock and financial metrics about S&P500 companies ",
    "snowflake_connection": snowpark,
}

## Python Tool Configuration

Configuring a Python Tool for the Agent Gateway requires 1) Python Callable 2) Tool Description (what does the tool do) 3) Output Description (what does the tool output). 

In the example below we create a NewsTool object that submits a HTTP request to a 3rd Party News API. The python callable is passed into the Python Tool as `news_api_func`.To use the tool below get your free token by signing up for an account at thenewsapi.com or just create your own python function and pass it into the PythonTool.

In [4]:
import requests
import json


class NewsTool:
    def __init__(self, token, limit) -> None:
        self.api_token = token
        self.limit = limit

    def news_search(self, news_query: str) -> str:
        news_request = f"""https://api.thenewsapi.com/v1/news/all?api_token={self.api_token}&search={news_query}&language=en&limit={self.limit}"""
        response = requests.get(news_request)
        json_response = json.loads(response.content)["data"]

        return str(json_response)


python_config = {
    "tool_description": "searches for relevant news based on user query",
    "output_description": "relevant articles",
    "python_func": NewsTool(token=os.getenv("NEWS_API_TOKEN"), limit=3).news_search,
}

## Agent Config

After the tools have been configured, initialize them and configure the agent.

In [ ]:
annual_reports = CortexSearchTool(**search_config)
sp500 = CortexAnalystTool(**analyst_config)
news_search = PythonTool(**python_config)

snowflake_tools = [annual_reports, sp500, news_search]
agent = Agent(snowflake_connection=snowpark, tools=snowflake_tools)

# Agent Testing

The 3 types of questions below are designed to showcase the breadth of tool use patterns possible with the Agent Gateway. 

- The Structured Data Questions use the Cortex Analyst agent. 
- The Unstructured Data Questions use either the Cortex Search agent or the Python (News API) agent.
- The last section includes a question that requires the use of both types of tools.

## Structured Data Questions

In [ ]:
agent("What is the market cap of Apple?")

In [ ]:
agent("Which company has the bigger EBITDA, Apple or MSFT?")

# Unstructured Data Questions

In [ ]:
agent("How many customers does Snowflake have?")

In [ ]:
agent("Give me recent news about Azure's AI investments")

## Unstructured + Structured Data Questions

In [ ]:
agent(
    "Which S&P500 companies are considered competitors in Snowflake's annual report? What is the market cap of each company mentioned?"
)

## With Observability

To add observability, use the Tru_Agent and pass in a TruLens/Snowflake Connector. This also allows you to log in different tables than the agent is connected to.

In [ ]:
from agent_gateway import TruAgent

from trulens.connectors.snowflake import SnowflakeConnector

trulens_snowflake_connection = SnowflakeConnector(**connection_parameters)

observed_agent = TruAgent(
    snowflake_connection=snowpark,
    tools=snowflake_tools,
    trulens_snowflake_connection=trulens_snowflake_connection,
    app_name="agent",
    app_version="v1",
)

In [ ]:
observed_agent("How many customers does Snowflake have?")

In [ ]:
observed_agent("What is the market cap of Apple?")

## View local TruLens dashboard

Or see tracing in Snowsight dashboard if opted in to AI Observability Preview

In [ ]:
from trulens.dashboard import run_dashboard

from trulens.core import TruSession

trusession = TruSession(connector=trulens_snowflake_connection)

run_dashboard(session=trusession)